## Set up google colab environment

In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os
os.chdir('/content/drive/My Drive/Hyperspectral_Image_Classification/')

In [2]:
from sample_extraction_V1_utils import *
import scipy.io as sio

## Load Indian Pines Dataset

In [3]:
uIndianPines = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Indian_pines_corrected.mat')
gt_IndianPines = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Indian_pines_gt.mat')

In [4]:
data = uIndianPines['indian_pines_corrected']
ground_truth = gt_IndianPines['indian_pines_gt']

In [5]:
data.shape

(145, 145, 200)

In [6]:
ground_truth.shape

(145, 145)

## Distrubution of samples for each class

In [7]:
class_distribution = pd.DataFrame(np.unique(ground_truth, return_counts = True))
class_distribution = class_distribution.transpose()
class_distribution.columns = ['class','samples']
class_distribution

,class,samples
0,0,10776
1,1,46
2,2,1428
3,3,830
4,4,237
5,5,483
6,6,730
7,7,28
8,8,478
9,9,20


In [8]:
classes , counts = np.unique(ground_truth, return_counts = True)
classes = classes[[2,3,5,6,8,10,11,12,14]] ## Dropping classes with small number of samples
classes

array([ 2,  3,  5,  6,  8, 10, 11, 12, 14], dtype=uint8)

## Source : Indian Pines

## Train model for samples extracted with overlap ratio 1 and varied number of samples picked from each class to be present in the training set. 

## Rest of the samples from each class are added to the test set.

## Model except the final fully connected layer is saved for transfer learning.

In [9]:
pretrain_results = pretrain_source_models(training_set_size = [200,250,300],
                                          classes = classes,
                                          cube_size = 20,
                                          overlap_ratio = 1,
                                          data = data,
                                          ground_truth = ground_truth,
                                          batch_size = 20,
                                          channels = 64,
                                          epochs = 100,
                                          Verbosity = 1,
                                          accuracies = [],
                                          learning_rate = 0.0001,
                                          source_dataset = 'indian_pines')


Model training starts for data with 200 samples from each class in training set

Samples per class: [1368, 523, 323, 730, 356, 837, 2224, 460, 1085]
Total number of samples 7906.

Total number of samples in training set 1800.
unique classes in training set: [ 2  3  5  6  8 10 11 12 14]
Samples per class in training set: [200 200 200 200 200 200 200 200 200]

Total number of samples in test set 6106.
unique classes in test set: [ 2  3  5  6  8 10 11 12 14]
Samples per class in test set: [1168  323  123  530  156  637 2024  260  885]

X_train => (1800, 20, 20, 64)
X_test  => (6106, 20, 20, 64)
y_train => (1800, 9)
y_test  => (6106, 9)

After shuffle : (None, 5, 5, 128)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20, 20, 64)] 0                                            
___________

In [10]:
pretrain_results

,Training Samples from each class,Training Samples,Test Samples,Test_Accuracies
0,200,1800,6106,48.919097
1,250,2250,5656,53.376943
2,300,2700,5206,59.930849
